# VacationPy

---

The goal is to find a vocation city or cities with ideal weather conditions based on the custom criteria using jupyter-gmaps and the Google Places API.

---

In [36]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension
#!jupyter nbextension enable --py --sys-prefix gmaps

In [39]:
# Dependencies and Setup:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key:
from api_keys import g_key

---
### Loading City Data From the Home Work Part 1

In [40]:
# File to Load:
file_to_load='../weatherpy/output_data/cities.csv' # csv file created in part I of homework 

# Read csv cities file and store into data frame:
city_data=pd.read_csv(file_to_load)
city_data.head()

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,Ushuaia,AR,1586644613,-54.80,-68.30,40,80,41.00,23.04
1,Ardatov,RU,1586644688,55.24,43.10,100,82,32.54,13.78
2,Puerto Ayora,EC,1586644688,-0.74,-90.35,40,70,84.20,9.17
3,Chichihualco,MX,1586644688,17.68,-99.65,32,18,85.15,6.38
4,Luganville,VU,1586644688,-15.53,167.17,40,69,82.40,6.93


In [41]:
city_data.count()

City                566
Country             561
Date                566
Lat                 566
Lng                 566
Cloudiness (%)      566
Humidity (%)        566
Max Temp (F)        566
Wind Speed (mph)    566
dtype: int64

---
### Humidity Heatmap

In [42]:
# Determan max humidity level in the city data frame:
max_hum=city_data['Humidity (%)'].max()
max_hum

100

In [43]:
# Configure gmaps with API key:
gmaps.configure(api_key=g_key)

In [44]:
# Store 'Lat' and 'Lng' into  locations: 
city_data=city_data.dropna() # drop NaN values
locations=city_data[['Lat', 'Lng']].astype(float)

# Convert humidity level to float and store:
humidity_rate=city_data['Humidity (%)'].astype(float)

In [46]:
# Create a humidity heat map layer:
fig=gmaps.figure()

heat_layer=gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=max_hum,
                                 point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

---
### New DataFrame Fitting Custom Weather Criteria

In [47]:
# Create new data frame with requested weather conditions:
city_data_narrow=city_data[(city_data['Max Temp (F)'] < 80) &
                             (city_data['Max Temp (F)'] > 65) &
                             (city_data['Wind Speed (mph)'] < 10) &
                             (city_data['Cloudiness (%)'] < 15) &
                             (city_data['Humidity (%)'] < 30)]
city_data_narrow=city_data_narrow.dropna() # drop any rows will null values
city_data_narrow.count()

City                6
Country             6
Date                6
Lat                 6
Lng                 6
Cloudiness (%)      6
Humidity (%)        6
Max Temp (F)        6
Wind Speed (mph)    6
dtype: int64

In [48]:
city_data_narrow.head(6)

,City,Country,Date,Lat,Lng,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
80,Mount Isa,AU,1586644695,-20.73,139.50,13,19,71.60,8.05
139,Tifton,US,1586644700,31.45,-83.51,1,27,77.00,6.93
292,Fort Morgan,US,1586644713,40.25,-103.80,1,10,73.40,5.03
368,Gīdam,IN,1586644719,18.98,81.40,0,27,73.65,2.53
440,Forest,US,1586644538,37.36,-79.29,1,20,66.99,8.05
454,Grand Island,US,1586644638,40.92,-98.34,1,25,73.40,9.17


In [49]:
# Set up lists to hold response info:
name=[]         # list for hotel name
city=[]         # list for hotel city
country=[]      # list for hotel country
lat=[]          # list for hotel latitude
lng=[]          # list for hotel longitude

# Loop through the list of cities and perform a request for data on each and add to corresponding lists:

count=1 # set counter to start value 1 for printouts of the current set count

print('Beginning Data Retrieval')     
print('-----------------------------')
for i in range(len(city_data_narrow)) : 
   
    lati=str(city_data_narrow.iloc[i, 3])    
    lngi=str(city_data_narrow.iloc[i, 4])
    
    target_coordinates=lati + ', ' + lngi  
    target_search='Hotel'
    target_radius=5000
    target_type='lodging'

    # Set up a parameters dictionary:
    params={
        'location': target_coordinates,
        'keyword': target_search,
        'radius': target_radius,
        'type': target_type,
        'key': g_key
    }

    # Base url:
    base_url='https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # Run a request using the above params dictionary:
    response=requests.get(base_url, params=params)
    #print(response.url)
    
    # Convert response to json:
    places_data=response.json()
    #print(json.dumps(places_data, indent=4, sort_keys=True))
    
    # Print user feedback: 
    print('Looking for Hotels near: ' + str(city_data_narrow.iloc[i, 0]) + ' | Record ' + str(count) + ' of Total ' + str(len(city_data_narrow)))
     
    # Add requested data to  result lists:
    try:                                             # ignore errors in response
        name.append(places_data['results'][0]['name'])
        city.append(city_data_narrow.iloc[i, 0])
        country.append(city_data_narrow.iloc[i, 1])
        lat.append(places_data['results'][0]['geometry']['location']['lat'])
        lng.append(places_data['results'][0]['geometry']['location']['lng'])
    except IndexError:
        print('No Hotel Near Location... Skipping...') # in case one of the responses comes back with error "IndexError"
                                                           # which could be no hotel near location 
  
    count=count+1
    
# Print Ending Log Statement:
print('-------------------------------')
print('Data Retrieval Complete')
print('-------------------------------')

Beginning Data Retrieval
-----------------------------
Looking for Hotels near: Mount Isa | Record 1 of Total 6
Looking for Hotels near: Tifton | Record 2 of Total 6
Looking for Hotels near: Fort Morgan | Record 3 of Total 6
Looking for Hotels near: Gīdam | Record 4 of Total 6
No Hotel Near Location... Skipping...
Looking for Hotels near: Forest | Record 5 of Total 6
No Hotel Near Location... Skipping...
Looking for Hotels near: Grand Island | Record 6 of Total 6
-------------------------------
Data Retrieval Complete
-------------------------------


In [50]:
# Create a data frame from api call list:
hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Hotel Name': name
}
hotel_df=pd.DataFrame(hotel_dict)
hotel_df.count()

City          4
Country       4
Lat           4
Lng           4
Hotel Name    4
dtype: int64

In [51]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Mount Isa,AU,-20.699471,139.486778,Abacus Motel
1,Tifton,US,31.447463,-83.534218,Hilton Garden Inn Tifton
2,Fort Morgan,US,40.264564,-103.773168,Comfort Inn
3,Grand Island,US,40.895797,-98.338922,Best Western Plus Grand Island Inn & Suites


---
### Hotel Map for Vacation Cities

In [52]:
# NOTE: Do not change any of the code in this cell.

# Using the template add the hotel marks to the heatmap:
info_box_template="""
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row:
# NOTE: be sure to update with your DataFrame name!
hotel_info=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations=hotel_df[['Lat', 'Lng']]

In [53]:
# Customize the size of the figure:
figure_layout={
    'width': '955px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig=gmaps.figure(layout=figure_layout)

# Assign the marker layer to a variable:
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map:
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [54]:
# Create a combined map:
fig=gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))